In [1]:
import pandas as pd
import re
import sys
import numpy as np
from ast import literal_eval as make_tuple
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.callbacks import History 
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Embedding, Input,BatchNormalization, Dense, Bidirectional,LSTM,Dropout,Activation, concatenate
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Embedding, Input,InputLayer,BatchNormalization, Dense, Bidirectional,LSTM,Dropout,GRU,Activation
from keras.backend import sigmoid

In [2]:
labelMapping = {'Component-Whole(e2,e1)': 0, 'Other': 1, 'Instrument-Agency(e2,e1)': 2, 'Member-Collection(e1,e2)': 3, 'Cause-Effect(e2,e1)': 4, 'Entity-Destination(e1,e2)': 5, 'Content-Container(e1,e2)': 6, 'Message-Topic(e1,e2)': 7, 'Product-Producer(e2,e1)': 8, 'Member-Collection(e2,e1)': 9, 'Entity-Origin(e1,e2)': 10, 'Cause-Effect(e1,e2)': 11, 'Component-Whole(e1,e2)': 12, 'Message-Topic(e2,e1)': 13, 'Product-Producer(e1,e2)': 14, 'Entity-Origin(e2,e1)': 15, 'Content-Container(e2,e1)': 16, 'Instrument-Agency(e1,e2)': 17, 'Entity-Destination(e2,e1)': 18}

In [3]:
df = pd.read_csv('./entityRelations/task2_train.csv')
df2 = pd.read_csv('./entityRelations/train_dataset.csv')
df_test = pd.read_csv('./entityRelations/task2_test.csv')
df2_test = pd.read_csv('./entityRelations/test_dataset.csv')

In [4]:
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

In [5]:
num_classes = 19
train_text = []
p1 = [make_tuple(x)[0] for x in df2['entity_indices']]
p2 = [make_tuple(x)[1] for x in df2['entity_indices']]

In [6]:
for i in range(len(df)):
    split_sentence = df.iloc[i]['text'].split(' ')
    prune_text = " ".join(split_sentence[p1[i]:p2[i]+1])
    train_text.append(prune_text) 

In [7]:
tokenizer = Tokenizer(num_words=25000,lower=True,split=' ',char_level=False)
tokenizer.fit_on_texts(train_text)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9889 unique tokens.


In [8]:
train_sentence_seq = tokenizer.texts_to_sequences(train_text)

In [9]:
max_length = np.max([len(i) for i in train_sentence_seq])
print("max length:", max_length)

max length: 33


In [10]:
x_train_seq = sequence.pad_sequences(train_sentence_seq, maxlen=max_length)
dependency_list = df['deps']
a, b = np.unique(dependency_list, return_counts=True)
a_sorted = a[np.argsort(b)[::-1]]
major_dep = a_sorted[:33]
major_dep

array(['nsubjpobj', 'nsubjpasspobj', 'dobjpobj', 'nsubjdobj', 'pobjpobj',
       'compoundpobj', 'attrpobj', 'compoundnsubj', 'conjpobj',
       'pobjdobj', 'nsubjcompound', 'compounddobj', 'dobjdobj',
       'nsubjnsubj', 'nsubjpunct', 'ROOTpobj', 'pobjcompound', 'attrdobj',
       'nsubjpasscompound', 'dobjcompound', 'nsubjconj',
       'compoundcompound', 'pobjnsubj', 'nsubjpassdobj', 'pobjconj',
       'nsubjpasspunct', 'nsubjamod', 'compoundnsubjpass', 'punctprep',
       'compoundattr', 'conjdobj', 'appospobj', 'pobjattr'], dtype=object)

In [11]:
dependency_list_filter = []
for dep in dependency_list:
    if dep in major_dep:
        dependency_list_filter.append(dep)
    else:
        dependency_list_filter.append("other")

In [12]:
def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r',encoding='utf-8')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel

In [13]:
from sklearn.preprocessing import LabelBinarizer

In [14]:
w2vModel = loadGloveModel('glove.42B.300d.txt')

Loading Glove Model
1917494  words loaded!


In [15]:
embedding_size = 300
num_words = len(word_index)+1
embedding_matrix = np.zeros((num_words, embedding_size))
oov = 0
for word, i in word_index.items():
    if word in w2vModel.keys():
        embedding_matrix[i] = w2vModel[word]
    else:
        oov+=1
print("OOV:",oov)  

OOV: 81


In [17]:
lb = LabelBinarizer()
dependency_list_filter = lb.fit_transform(dependency_list_filter)

In [18]:
test_text = []

p1_test = [make_tuple(x)[0] for x in df2_test['entity_indices']]
p2_test = [make_tuple(x)[1] for x in df2_test['entity_indices']]

for i in range(len(df_test)):
    split_sentence = df_test.iloc[i]['text'].split(' ')
    prune_text = " ".join(split_sentence[p1_test[i]:p2_test[i]+1])
    test_text.append(prune_text) 
	
test_sentence_seq = tokenizer.texts_to_sequences(test_text)

x_test_seq = sequence.pad_sequences(test_sentence_seq, maxlen=max_length)

dependency_list_test = df_test['deps']

dependency_list_filter_test = []
for dep in dependency_list_test:
    if dep in major_dep:
        dependency_list_filter_test.append(dep)
    else:
        dependency_list_filter_test.append("other")
		
dependency_list_filter_test = lb.fit_transform(dependency_list_filter_test)

In [19]:
train_mut_ancestors_list = [make_tuple(x) for x in df['ancestors']]
test_mut_ancestors_list = [make_tuple(x) for x in df_test['ancestors']]
train_label = [labelMapping[ele] for ele in df2['labels']]
train_label = dense_to_one_hot(np.array(train_label), 19)
test_label = [labelMapping[ele] for ele in df2_test['labels']]
test_label = dense_to_one_hot(np.array(test_label), 19)
x_test_seq = sequence.pad_sequences(test_sentence_seq, maxlen=max_length)

In [22]:
def swish(x):
    return (sigmoid(x) * x)
get_custom_objects().update({'swish': Activation(swish)})


def train_BiLSTM(x_train, ancestor_train, dep_train, y_train, 
                 x_val, ancestor_val, dep_val, y_val,
                 embedding_matrix, max_length, max_features):
    embedding_size = 300
    batch_size = 64
    epochs = 50
    embedding_layer = Embedding(max_features,output_dim= embedding_size,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)
    sequence_input = Input(shape=(max_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    lstm0 = Bidirectional(LSTM(256,activation="tanh",dropout=0.2,return_sequences = True,
                kernel_initializer='he_uniform'))(embedded_sequences)
    lstm1 = Bidirectional(LSTM(128,activation="tanh",dropout=0.2,return_sequences = True,
                kernel_initializer='he_uniform'))(lstm0)
    lstm2 = Bidirectional(LSTM(64,activation="tanh",dropout=0.2,return_sequences = False,
                kernel_initializer='he_uniform'))(lstm1)
    bn1 = BatchNormalization()(lstm2)
    
    # other feature inputs 
    ancestor_input = Input(shape=(2,))
    ancestor_feature = Dense(64, activation=swish)(ancestor_input)
    
    
    dep_input = Input(shape=(34,))
    dep_feature = Dense(128, activation=swish)(dep_input)
    
    combine_feature = concatenate([bn1, ancestor_feature, dep_feature])
    dense1 = Dense(64, activation=swish)(combine_feature)
    dropout1 = Dropout(0.5)(dense1)
    dense2 = Dense(32, activation=swish)(dropout1)
    dropout2 = Dropout(0.5)(dense2)
    preds = Dense(19, activation='softmax')(dropout2)
    model = Model([sequence_input, ancestor_input, dep_input], preds)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    filepath = "models/BiLSTM_3.hdf5" 
    checkpoint = ModelCheckpoint(filepath,monitor='val_acc',save_best_only=True)
    history = History()
    callbacks_list = [checkpoint, history]
    
    history = model.fit([x_train, ancestor_train, dep_train], y_train, 
                        validation_data=([x_val, ancestor_val, dep_val], y_val), 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        callbacks=callbacks_list)
    return model, history

In [23]:
model, history = train_BiLSTM(np.asarray(x_train_seq), np.asarray(train_mut_ancestors_list), np.asarray(dependency_list_filter), np.asarray(train_label), 
                     np.asarray(x_test_seq), np.asarray(test_mut_ancestors_list), np.asarray(dependency_list_filter_test), np.asarray(test_label),
                     embedding_matrix,
                     max_length,
                     len(word_index)+1)

Epoch 1/50
125/125 [==============================] - 60s 479ms/step - loss: 2.3628 - acc: 0.3038 - val_loss: 2.1959 - val_acc: 0.4785
Epoch 2/50
125/125 [==============================] - 61s 487ms/step - loss: 1.7981 - acc: 0.4790 - val_loss: 1.4757 - val_acc: 0.6456
Epoch 3/50
125/125 [==============================] - 60s 481ms/step - loss: 1.4936 - acc: 0.5617 - val_loss: 1.0913 - val_acc: 0.6956
Epoch 4/50
125/125 [==============================] - 65s 520ms/step - loss: 1.2984 - acc: 0.6170 - val_loss: 1.0436 - val_acc: 0.7015
Epoch 5/50
125/125 [==============================] - 68s 545ms/step - loss: 1.1632 - acc: 0.6668 - val_loss: 0.9701 - val_acc: 0.7173
Epoch 6/50
125/125 [==============================] - 69s 549ms/step - loss: 1.0284 - acc: 0.7015 - val_loss: 0.9992 - val_acc: 0.7184
Epoch 7/50
125/125 [==============================] - 66s 532ms/step - loss: 0.9252 - acc: 0.7384 - val_loss: 1.0166 - val_acc: 0.7188
Epoch 8/50
125/125 [==============================] - 6

In [ ]:
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

prediction = model.predict([np.asarray(x_test_seq), np.asarray(test_mut_ancestors_list), np.asarray(dependency_list_filter_test)], batch_size = 1000)
class_pred = np.argmax(predict1ion, axis=1)
class_true = np.argmax(test_label, axis=1)
conf = confusion_matrix(class_true, class_pred)
precision_recall_fscore = precision_recall_fscore_support(class_true, class_pred, average='macro')